In [1]:
# Data Cleaning and Preprocessing - Task 1
# Data Analyst Internship

# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

print("Libraries imported successfully!")

# =============================================================================
# STEP 1: LOAD AND EXPLORE THE DATASET
# =============================================================================

# Load the dataset (replace 'dataset.csv' with your actual filename)
df = pd.read_csv('Mall_Customers.csv')  # Adjust filename as needed

print("Dataset loaded successfully!")
print(f"Dataset shape: {df.shape}")

# Display first few rows
print("\n" + "="*50)
print("FIRST 5 ROWS OF THE DATASET:")
print("="*50)
df.head()

# Display basic information about the dataset
print("\n" + "="*50)
print("DATASET INFORMATION:")
print("="*50)
df.info()

# Display basic statistics
print("\n" + "="*50)
print("BASIC STATISTICS:")
print("="*50)
df.describe()

# Check column names
print("\n" + "="*50)
print("COLUMN NAMES:")
print("="*50)
print("Original columns:", df.columns.tolist())

# =============================================================================
# STEP 2: IDENTIFY DATA QUALITY ISSUES
# =============================================================================

print("\n" + "="*60)
print("DATA QUALITY ASSESSMENT")
print("="*60)

# Check for missing values
print("\n1. MISSING VALUES:")
missing_values = df.isnull().sum()
print(missing_values)
print(f"Total missing values: {missing_values.sum()}")

# Check for duplicate rows
print("\n2. DUPLICATE ROWS:")
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Check data types
print("\n3. DATA TYPES:")
print(df.dtypes)

# Check unique values for categorical columns
print("\n4. UNIQUE VALUES IN CATEGORICAL COLUMNS:")
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print(f"\n{col}:")
    print(f"  Unique values: {df[col].nunique()}")
    print(f"  Values: {df[col].unique()}")

# Check for potential outliers in numerical columns
print("\n5. POTENTIAL OUTLIERS:")
numerical_cols = df.select_dtypes(include=[np.number]).columns
for col in numerical_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"{col}: {len(outliers)} potential outliers")

# =============================================================================
# STEP 3: DATA CLEANING OPERATIONS
# =============================================================================

print("\n" + "="*60)
print("STARTING DATA CLEANING PROCESS")
print("="*60)

# Create a copy of the original dataset for cleaning
df_original = df.copy()
df_cleaned = df.copy()

# Track changes
changes_made = []

# 3.1 Clean Column Names
print("\n3.1 CLEANING COLUMN NAMES:")
print("Before:", df_cleaned.columns.tolist())

# Standardize column names: lowercase, replace spaces with underscores
df_cleaned.columns = df_cleaned.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
df_cleaned.columns = df_cleaned.columns.str.replace('[^a-zA-Z0-9_]', '', regex=True)

print("After:", df_cleaned.columns.tolist())
changes_made.append("Standardized column names (lowercase, underscores)")

# 3.2 Handle Missing Values
print("\n3.2 HANDLING MISSING VALUES:")
if df_cleaned.isnull().sum().sum() > 0:
    print("Missing values found, handling them...")
    
    # For numerical columns: fill with median
    for col in df_cleaned.select_dtypes(include=[np.number]).columns:
        if df_cleaned[col].isnull().sum() > 0:
            median_value = df_cleaned[col].median()
            df_cleaned[col].fillna(median_value, inplace=True)
            changes_made.append(f"Filled missing values in {col} with median ({median_value})")
    
    # For categorical columns: fill with mode
    for col in df_cleaned.select_dtypes(include=['object']).columns:
        if df_cleaned[col].isnull().sum() > 0:
            mode_value = df_cleaned[col].mode()[0] if len(df_cleaned[col].mode()) > 0 else 'Unknown'
            df_cleaned[col].fillna(mode_value, inplace=True)
            changes_made.append(f"Filled missing values in {col} with mode ({mode_value})")
else:
    print("No missing values found!")

# 3.3 Remove Duplicates
print("\n3.3 REMOVING DUPLICATES:")
if duplicates > 0:
    df_cleaned = df_cleaned.drop_duplicates()
    print(f"Removed {duplicates} duplicate rows")
    changes_made.append(f"Removed {duplicates} duplicate rows")
else:
    print("No duplicate rows found!")

# 3.4 Standardize Text Data
print("\n3.4 STANDARDIZING TEXT DATA:")
for col in df_cleaned.select_dtypes(include=['object']).columns:
    # Remove leading/trailing whitespace and standardize case
    df_cleaned[col] = df_cleaned[col].astype(str).str.strip()
    
    # If it's a gender-like column, standardize values
    if 'gender' in col.lower():
        df_cleaned[col] = df_cleaned[col].str.lower()
        df_cleaned[col] = df_cleaned[col].replace({
            'm': 'male', 'f': 'female', 'male ': 'male', 'female ': 'female',
            'man': 'male', 'woman': 'female'
        })
        changes_made.append(f"Standardized gender values in {col}")
    
    print(f"Cleaned text in column: {col}")

# 3.5 Fix Data Types
print("\n3.5 FIXING DATA TYPES:")
# This section will depend on your specific dataset
# Common fixes:
for col in df_cleaned.columns:
    if 'id' in col.lower() and df_cleaned[col].dtype == 'float64':
        df_cleaned[col] = df_cleaned[col].astype('int64')
        changes_made.append(f"Converted {col} to integer")
    elif 'age' in col.lower() and df_cleaned[col].dtype == 'float64':
        df_cleaned[col] = df_cleaned[col].astype('int64')
        changes_made.append(f"Converted {col} to integer")

# 3.6 Handle Date Columns (if any exist)
print("\n3.6 HANDLING DATE COLUMNS:")
date_columns = [col for col in df_cleaned.columns if 'date' in col.lower() or 'time' in col.lower()]
if date_columns:
    for col in date_columns:
        try:
            df_cleaned[col] = pd.to_datetime(df_cleaned[col])
            changes_made.append(f"Converted {col} to datetime format")
            print(f"Converted {col} to datetime")
        except:
            print(f"Could not convert {col} to datetime")
else:
    print("No date columns found")

# =============================================================================
# STEP 4: QUALITY VALIDATION
# =============================================================================

print("\n" + "="*60)
print("POST-CLEANING VALIDATION")
print("="*60)

# Check for remaining issues
print("1. MISSING VALUES AFTER CLEANING:")
print(df_cleaned.isnull().sum())

print("\n2. DUPLICATE ROWS AFTER CLEANING:")
print(f"Duplicate rows: {df_cleaned.duplicated().sum()}")

print("\n3. DATA TYPES AFTER CLEANING:")
print(df_cleaned.dtypes)

print("\n4. FINAL DATASET SHAPE:")
print(f"Original shape: {df_original.shape}")
print(f"Cleaned shape: {df_cleaned.shape}")

# =============================================================================
# STEP 5: SAVE CLEANED DATASET
# =============================================================================

print("\n" + "="*60)
print("SAVING CLEANED DATASET")
print("="*60)

# Save the cleaned dataset
cleaned_filename = 'cleaned_dataset.csv'
df_cleaned.to_csv(cleaned_filename, index=False)
print(f"Cleaned dataset saved as: {cleaned_filename}")

# =============================================================================
# STEP 6: SUMMARY REPORT
# =============================================================================

print("\n" + "="*60)
print("DATA CLEANING SUMMARY REPORT")
print("="*60)

print(f"Original dataset: {df_original.shape[0]} rows, {df_original.shape[1]} columns")
print(f"Cleaned dataset: {df_cleaned.shape[0]} rows, {df_cleaned.shape[1]} columns")
print(f"Rows removed: {df_original.shape[0] - df_cleaned.shape[0]}")

print("\nChanges made during cleaning:")
for i, change in enumerate(changes_made, 1):
    print(f"{i}. {change}")

print("\nFinal data quality:")
print(f"- Missing values: {df_cleaned.isnull().sum().sum()}")
print(f"- Duplicate rows: {df_cleaned.duplicated().sum()}")
print(f"- Data types: {dict(df_cleaned.dtypes.value_counts())}")

print("\n" + "="*60)
print("DATA CLEANING COMPLETED SUCCESSFULLY!")
print("="*60)

# Display final cleaned dataset sample
print("\nFinal cleaned dataset (first 10 rows):")
df_cleaned.head(10)

Libraries imported successfully!
Dataset loaded successfully!
Dataset shape: (200, 5)

FIRST 5 ROWS OF THE DATASET:

DATASET INFORMATION:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CustomerID              200 non-null    int64 
 1   Gender                  200 non-null    object
 2   Age                     200 non-null    int64 
 3   Annual Income (k$)      200 non-null    int64 
 4   Spending Score (1-100)  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB

BASIC STATISTICS:

COLUMN NAMES:
Original columns: ['CustomerID', 'Gender', 'Age', 'Annual Income (k$)', 'Spending Score (1-100)']

DATA QUALITY ASSESSMENT

1. MISSING VALUES:
CustomerID                0
Gender                    0
Age                       0
Annual Income (k$)        0
Spending Score (1-100)    0
dtype: int64
Total missing 

,customerid,gender,age,annual_income_k,spending_score_1_100
0,1,male,19,15,39
1,2,male,21,15,81
2,3,female,20,16,6
3,4,female,23,16,77
4,5,female,31,17,40
5,6,female,22,17,76
6,7,female,35,18,6
7,8,female,23,18,94
8,9,male,64,19,3
9,10,female,30,19,72
